In [1]:
!pip install rouge_score tensorboard datasets evaluate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.8 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=0031a411ec0253755d550030dbcbb0e882de76023140cdbc971c042359e2841b
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dep

In [2]:
import gdown

# Extract file ID from your URL
file_id = "1WEr2yudayZNfuBSp0Y1q_pPI_xKuXEgJ"
download_url = f"https://drive.google.com/uc?id={file_id}"

# Download the file
output = "legal_summaries.csv"
gdown.download(download_url, output, quiet=False,fuzzy=True)


Downloading...
From (original): https://drive.google.com/uc?id=1WEr2yudayZNfuBSp0Y1q_pPI_xKuXEgJ
From (redirected): https://drive.google.com/uc?id=1WEr2yudayZNfuBSp0Y1q_pPI_xKuXEgJ&confirm=t&uuid=b07fe61a-e8a7-4de6-a999-6bb4b34d9d05
To: /content/legal_summaries.csv
100%|██████████| 277M/277M [00:05<00:00, 48.5MB/s]


'legal_summaries.csv'

In [3]:
import pandas as pd
data=pd.read_csv("legal_summaries.csv")
data.head()

,input_text,summary_text
0,Appeal No. LXVI of 1949.\nAppeal from the High...,The charge created in respect of municipal pro...
1,Civil Appeal No.94 of 1949.\n107 834 Appeal fr...,"An agreement for a lease, which a lease is by ..."
2,"iminal Appeal No. 40 of 1951, 127 Appeal from ...","The question whether a Magistrate is ""personal..."
3,Appeal No. 388 of 1960.\nAppeal by special lea...,The appellant was a member of a joint Hindu fa...
4,Appeal No. 198 of 1954.\nAppeal from the judgm...,The appellant was the Ruler of the State of Ba...


In [4]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

class LegalSummarizer:
    def __init__(self, model_name="t5-base"):
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)

    def summarize(self, document, max_length=250, min_length=100):
        inputs = self.tokenizer(
            f"summarize: {document}",
            max_length=1024,
            truncation=True,
            return_tensors="pt"
        )

        summary_ids = self.model.generate(
            inputs['input_ids'],
            max_length=max_length,
            min_length=min_length,
            num_beams=4,
            early_stopping=True
        )

        return self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Usage
long_legal_document=data['input_text'][0]
summarizer = LegalSummarizer()
summary = summarizer.summarize(long_legal_document)
print(summary)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

munici pal property tax and urban immoveable property tax are allowable deductions under section 9 (1) (iv) of the Indian Income tax Act . the assessee company paid in the relevant year Rs. 1,22,675 as municipal property tax and Rs. 32,760 as urban property tax . the appeals of the assessee to the Appellate As sistant Commissioner and to the Income tax Appellate Tribu nal were unsuccessful .


In [9]:
from typing import Union, Dict, List
import pandas as pd
import torch
import numpy as np
import gc
from datasets import Dataset
import evaluate
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)

class MemoryOptimizedTrainer:
    def __init__(self, model_name="t5-base", cache_dir=None):
        self.model = T5ForConditionalGeneration.from_pretrained(
            model_name,
            cache_dir=cache_dir,
            use_cache=False
        )
        self.model.gradient_checkpointing_enable()

        # Initialize tokenizer with special token handling
        self.tokenizer = T5Tokenizer.from_pretrained(
            model_name,
            cache_dir=cache_dir,
            model_max_length=512,
            truncation_side='left'
        )

        self.device = torch.device("cpu")
        self.model.to(self.device)

        self.rouge = evaluate.load('rouge')

        # Update data collator with explicit padding token
        self.data_collator = DataCollatorForSeq2Seq(
            tokenizer=self.tokenizer,
            model=self.model,
            padding='longest',
            max_length=512,
            pad_to_multiple_of=8
        )

    def prepare_dataset(self, data: Union[pd.DataFrame, List[Dict]], test_size: float = 0.1,
                       max_samples: int = None) -> tuple:
        if not isinstance(data, pd.DataFrame):
            data = pd.DataFrame(data)

        if max_samples and len(data) > max_samples:
            data = data.sample(n=max_samples, random_state=42)

        # Enhanced data cleaning
        clean_data = data.copy()
        for col in ['input_text', 'summary_text']:
            clean_data[col] = (clean_data[col]
                             .fillna('')
                             .astype(str)
                             .str.strip()
                             .str.replace('\n', ' ')
                             .str.replace('\r', ' ')
                             .str.replace('\t', ' ')
                             .str.replace(r'\s+', ' ', regex=True))

        # Filter out empty entries
        clean_data = clean_data[
            (clean_data['input_text'].str.len() > 0) &
            (clean_data['summary_text'].str.len() > 0)
        ]

        clean_data = clean_data.rename(columns={'summary_text': 'target_text'})
        dataset = Dataset.from_pandas(clean_data)
        splits = dataset.train_test_split(test_size=test_size, shuffle=True, seed=42)

        return splits['train'], splits['test']

    def preprocess_batch(self, examples, max_input_length: int = 512,
                        max_target_length: int = 128):
        # Add prefix to input
        inputs = [f"summarize legal text: {doc}" for doc in examples['input_text']]

        # Tokenize with careful handling of special tokens
        model_inputs = self.tokenizer(
            inputs,
            max_length=max_input_length,
            truncation=True,
            padding=False,
            return_tensors=None,
            add_special_tokens=True
        )

        # Tokenize targets with special token handling
        labels = self.tokenizer(
            examples['target_text'],
            max_length=max_target_length,
            truncation=True,
            padding=False,
            return_tensors=None,
            add_special_tokens=True
        )

        model_inputs['labels'] = labels['input_ids']

        # Verify input_ids and labels
        for idx, (input_ids, label) in enumerate(zip(model_inputs['input_ids'], model_inputs['labels'])):
            if not all(0 <= id < self.tokenizer.vocab_size for id in input_ids):
                print(f"Warning: Invalid input_ids in batch at position {idx}")
            if not all(id == -100 or 0 <= id < self.tokenizer.vocab_size for id in label):
                print(f"Warning: Invalid label in batch at position {idx}")

        return model_inputs

    def compute_metrics(self, eval_pred):
        predictions, labels = eval_pred

        if len(predictions) == 0 or len(labels) == 0:
            print("Warning: Empty predictions or labels")
            return {'rouge1': 0.0, 'rouge2': 0.0, 'rougel': 0.0, 'rougelsum': 0.0}

        batch_size = 32
        decoded_preds = []
        decoded_labels = []

        try:
            for i in range(0, len(predictions), batch_size):
                batch_preds = predictions[i:i + batch_size]
                batch_labels = labels[i:i + batch_size]

                # Carefully decode predictions
                batch_decoded_preds = self.tokenizer.batch_decode(
                    batch_preds,
                    skip_special_tokens=True,
                    clean_up_tokenization_spaces=True
                )
                decoded_preds.extend(batch_decoded_preds)

                # Handle labels with care
                valid_labels = np.where(batch_labels != -100, batch_labels, self.tokenizer.pad_token_id)
                batch_decoded_labels = self.tokenizer.batch_decode(
                    valid_labels,
                    skip_special_tokens=True,
                    clean_up_tokenization_spaces=True
                )
                decoded_labels.extend(batch_decoded_labels)

            # Clean up decoded text
            decoded_preds = [pred.strip() for pred in decoded_preds if pred.strip()]
            decoded_labels = [label.strip() for label in decoded_labels if label.strip()]

            # Ensure we have valid pairs
            if not decoded_preds or not decoded_labels:
                print("Warning: No valid prediction-label pairs after decoding")
                return {'rouge1': 0.0, 'rouge2': 0.0, 'rougel': 0.0, 'rougelsum': 0.0}

            # Compute ROUGE scores
            result = self.rouge.compute(
                predictions=decoded_preds,
                references=decoded_labels,
                use_stemmer=True
            )

            return {key: round(value * 100, 2) for key, value in result.items()}

        except Exception as e:
            print(f"Error in compute_metrics: {str(e)}")
            return {'rouge1': 0.0, 'rouge2': 0.0, 'rougel': 0.0, 'rougelsum': 0.0}

    def fine_tune(self, data: Union[pd.DataFrame, List[Dict]],
                 output_dir: str = './legal_summarizer_model',
                 num_epochs: int = 3,
                 batch_size: int = 2,
                 max_samples: int = 10000,
                 gradient_accumulation_steps: int = 32):

        train_dataset, val_dataset = self.prepare_dataset(data, max_samples=max_samples)

        # Add error handling for dataset preprocessing
        try:
            train_dataset = train_dataset.map(
                self.preprocess_batch,
                batched=True,
                batch_size=100,
                remove_columns=train_dataset.column_names,
                num_proc=1  # Reduced for stability
            )

            val_dataset = val_dataset.map(
                self.preprocess_batch,
                batched=True,
                batch_size=100,
                remove_columns=val_dataset.column_names,
                num_proc=1
            )
        except Exception as e:
            print(f"Error preprocessing dataset: {str(e)}")
            raise

        training_args = Seq2SeqTrainingArguments(
            output_dir=output_dir,
            num_train_epochs=num_epochs,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            eval_accumulation_steps=gradient_accumulation_steps,
            eval_steps=50,
            save_steps=50,
            logging_steps=25,
            learning_rate=2e-5,
            warmup_ratio=0.15,
            weight_decay=0.02,
            fp16=True,
            gradient_checkpointing=True,
            save_total_limit=2,
            predict_with_generate=True,
            generation_max_length=128,
            generation_num_beams=4,
            load_best_model_at_end=True,
            metric_for_best_model="rouge1",
            greater_is_better=True,
            evaluation_strategy="steps",
            remove_unused_columns=False,
            max_grad_norm=0.5,
            lr_scheduler_type="cosine",
            seed=42
        )

        trainer = Seq2SeqTrainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            data_collator=self.data_collator,
            compute_metrics=self.compute_metrics,
            tokenizer=self.tokenizer,
        )

        gc.collect()
        torch.cuda.empty_cache()

        if torch.cuda.is_available():
            self.device = torch.device("cuda")
            self.model.to(self.device)

        try:
            trainer.train()
            trainer.save_model(f'{output_dir}/final')
            self.tokenizer.save_pretrained(f'{output_dir}/final')

        except Exception as e:
            print(f"Error during training: {str(e)}")
            raise

        finally:
            self.model.to("cpu")
            gc.collect()
            torch.cuda.empty_cache()

In [11]:
trainer = MemoryOptimizedTrainer()
trainer.fine_tune(
    data,
    output_dir='./legal_summarizer_model',
    batch_size=2,
    gradient_accumulation_steps=32,
    max_samples=5000  # Optionally reduce dataset size for testing
)

Map:   0%|          | 0/4498 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-d24df9d9fc67>:229: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
50,3.504700,2.540773,0.000000,0.000000,0.000000,0.000000
100,2.737700,2.308180,0.000000,0.000000,0.000000,0.000000
150,2.439600,2.253435,0.000000,0.000000,0.000000,0.000000
200,2.405800,2.240903,0.000000,0.000000,0.000000,0.000000


Error in compute_metrics: piece id is out of range.
Error in compute_metrics: piece id is out of range.
Error in compute_metrics: piece id is out of range.
Error in compute_metrics: piece id is out of range.


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


In [17]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the trained model and tokenizer
model_path = './legal_summarizer_model/final'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

def generate_summary(text):
    # Prepare the input
    input_text = f"summarize legal text: {text}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate summary
    summary_ids = model.generate(
        inputs.input_ids,
        max_length=300,
        num_beams=4,
        length_penalty=2.0,
        early_stopping=True
    )

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Example usage
legal_text = data['input_text'][0]
summary = generate_summary(legal_text)
print(summary)

the learned Attorney General in view of these decisions did not support the view expressed by Kania J. Reliance was placed on a decision of the Allahabad High Court of Madras in Mamad Keyi vs Commissioner of Income tax, Bombay(3), in which moneys paid as urban immoveable property tax under the Bombay Finance Act were disallowed as inadmissible under section 9 (1) (iv) or 9 (1) (v) of the Indian Income tax Act. The learned Attorney General in view of the decision.


In [14]:
from google.colab import drive
drive.mount('/content/drive')

# Define the path in your Google Drive where you want to save the model
drive_path = "/content/drive/My Drive/legal_summarizer_model_final"  # You can change this path

# Save model and tokenizer to Google Drive
model.save_pretrained(drive_path)
tokenizer.save_pretrained(drive_path)

Mounted at /content/drive


('/content/drive/My Drive/legal_summarizer_model_final/tokenizer_config.json',
 '/content/drive/My Drive/legal_summarizer_model_final/special_tokens_map.json',
 '/content/drive/My Drive/legal_summarizer_model_final/spiece.model',
 '/content/drive/My Drive/legal_summarizer_model_final/added_tokens.json')

In [16]:
print(data['summary_text'][0])

The charge created in respect of municipal property tax by section 212 of the City of Bombay Municipal Act, 1888, is an "annual charge not being a capital charge" within the mean ing of section 9 (1) (iv) of the Indian Income tax Act, 199.2, and the amount of such charge should therefore be deducted in computing the income from such property for the purposes of section 9 of the Indian Income tax Act.
The charge in respect of urban immoveable property tax created by the Bombay Finance Act, 1939 is similar in character and the amount of such charge should also be deducted.
The expression "capital charge" in s.9(1) (iv) means a charge created for a capital sum,that is to say, a charge created to. ' secure the discharge of a liability of a capi tal nature; and an "annual charge" means a charge to secure an annual liabili ty. 554



In [19]:
# Load from Google Drive
drive_path = "/content/drive/My Drive/legal_summarizer_model_final"
loaded_model = T5ForConditionalGeneration.from_pretrained(drive_path)
loaded_tokenizer = T5Tokenizer.from_pretrained(drive_path)

In [23]:
def generate_focused_summary(text):
    # Preprocess to focus on key legal points
    input_text = f"summarize legal text focusing on main legal principle and tax classification: {text}"

    inputs = loaded_tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

    summary_ids = loaded_model.generate(
        inputs.input_ids,
        max_length=200,          # Even longer max length
        min_length=100,          # Force longer summary
        num_beams=8,             # More beams for better search
        length_penalty=2.0,      # Encourage longer outputs
        early_stopping=False,    # Don't stop early
        no_repeat_ngram_size=4,  # Stricter repetition prevention
        temperature=0.8,         # More randomness
        top_k=50,               # More diverse outputs
        do_sample=True          # Enable sampling
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Clean and format the input text
legal_text = data['input_text'][0]

print("New Summary with Modified Parameters:")
new_summary = generate_focused_summary(legal_text)
print(new_summary)

New Summary with Modified Parameters:
the learned Attorney General in view of these decisions did not support the view expressed by Kania J. Reliance was placed on a decision of the Allahabad High Court of Madras in Mamad Keyi vs Commissioner of Income tax, Bombay(3), in which moneys paid as urban immoveable property tax under the Bombay Finance Act were disallowed as inadmissible under section 9 (1) (iv) or 9 (1) (v) of the Indian Income tax Act. The learned Attorney General.


In [28]:
from transformers import pipeline
import re

def clean_text(text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)  # Remove non-ASCII characters
    text = re.sub(r"\s+", " ", text)  # Normalize whitespace
    return text

summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device="cpu", truncation=True)  # Change device to "cpu" and add truncation
legal_text = data['input_text'][0]
cleaned_text = clean_text(legal_text)  # Clean the input text
summary = summarizer(cleaned_text, max_length=150, min_length=50, do_sample=False)  # Use cleaned text
print(summary[0]['summary_text'])

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


IndexError: index out of range in self